In [44]:
import os
from py2neo import Graph, Node, Relationship
from tqdm import tqdm



In [45]:

user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")

graph_db: Graph = Graph("bolt://localhost:7687", auth=(user, password))

### lista para o algoritmos ###
* passar uma lista de pontos de interesse junto com o ponto inicial 
* calcular a distancia relativa entre os pontos
* testar todas as possibilidades de caminhos ate encontrar o menor (força bruta)

In [46]:
#achar a rua mais perto do ponto passado
def findRoadById(id: int):
    query = f'''
    MATCH  (n: Node) where id(n) = {id} return n
    '''
    return graph_db.run(query)

def findRoadId(latitude: float, longitude: float):
    query = f'''
    WITH {latitude} AS targetLat, {longitude} AS targetLon  

    MATCH (a)-[r:ROAD_TO]->(b)
    WHERE a.latitude IS NOT NULL AND a.longitude IS NOT NULL
    AND b.latitude IS NOT NULL AND b.longitude IS NOT NULL

    WITH r, a, b,
        point({{y: a.latitude, x: a.longitude}}) AS pointA,
        point({{y: b.latitude, x: b.longitude}}) AS pointB,
        point({{y: targetLat, x: targetLon}}) AS targetPoint

    WITH r, a, b,
        point.distance(pointA, targetPoint) AS distA,
        point.distance(pointB, targetPoint) AS distB

    WITH r, a, b,
        CASE WHEN distA < distB THEN distA ELSE distB END AS minDist

    ORDER BY minDist ASC
    LIMIT 1

    RETURN id(r)
    '''
    id =  graph_db.run(query)
    return id.closed

def shorPath(paramA: int, paramB: int):
    query = f'''
    MATCH (start), (end)
    WHERE id(start) = {paramA} AND id(end) = {paramB}
    MATCH path = shortestPath((start)-[rels:ROAD_TO*]->(end))
    RETURN path,reduce(total = 0, r IN rels | total + r.length) AS totalLength
    '''
    return graph_db.run(query)


In [47]:
# achar o ponto mais perto entre os pontos de interesse


In [52]:
initialRoad = findRoadId(latitude=-23.2848682, longitude=-47.6720885)

target_points = [
    findRoadId(latitude=-23.2999866, longitude=-47.6650897),
    findRoadId(latitude=-23.2829741, longitude=-47.6745936),
    findRoadId(latitude=-23.2847718, longitude=-47.6730844)
]

target_points.append(initialRoad)
result = []
for id in target_points:
    for subid in target_points:
        if id != subid:
            print(id, subid)
            result.append(shorPath(id, subid))
            
print(result)
 

[]
